# Experiment - detekcia s použitím informácie o vzdialenosti

In [1]:
import dataset, model, evaluate
from tqdm import tqdm

Initializing the data manager
Data manager initialization finished


In [3]:
data_set = dataset.load_data_set('../res/cache/datasets/dataset_1000')

## Setting: lines

In [9]:
clip_line_distances = []
for data_point in tqdm(data_set):
    clip_line_distances.append(model.compute_clip_lines_dst(data_point))

100%|█████████▉| 990/991 [11:39<00:00,  1.42it/s]  


In [11]:
results = evaluate.eval_threshold_data_set(data_set, clip_line_distances)

In [29]:
precision, recall, f1_score = evaluate.calculate_metrics(*results)
print(f"{precision=:.3f} {recall=:.3f} {f1_score=:.3f}")

precision=0.301 recall=0.281 f1_score=0.291


In [25]:
adjusted_score = [evaluate.adjust_score_by_dist(clip_res, dist_far_penalty=0.00001, dist_near_threshold=10000) for clip_res in clip_line_distances]

In [26]:
results2 = evaluate.eval_threshold_data_set(data_set, adjusted_score)

In [30]:
precision, recall, f1_score = evaluate.calculate_metrics(*results2)
print(f"{precision=:.3f} {recall=:.3f} {f1_score=:.3f}")

precision=0.386 recall=0.308 f1_score=0.342


## Setting: regions

In [3]:
clip_region_distances = []
for data_point in tqdm(data_set):
    clip_region_distances.append(model.compute_clip_regions_dst(data_point))

  0%|          | 0/991 [00:00<?, ?it/s]C:\Users\H492635\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
100%|█████████▉| 990/991 [01:43<00:00,  9.56it/s]


In [4]:
reg_results = evaluate.eval_region_data_set(data_set, clip_region_distances)

In [5]:
precision, recall, f1_score = evaluate.calculate_metrics(*reg_results)
print(f"{precision=:.3f} {recall=:.3f} {f1_score=:.3f}")

precision=0.502 recall=0.645 f1_score=0.564


In [14]:
adjusted_reg_score = [evaluate.adjust_score_by_dist(clip_res, len_max=1000, dist_far_penalty=0.00001, dist_near_threshold=10000) for clip_res in clip_region_distances]

In [15]:
adjusted_reg_results = evaluate.eval_region_data_set(data_set, adjusted_reg_score)

In [16]:
precision, recall, f1_score = evaluate.calculate_metrics(*adjusted_reg_results)
print(f"{precision=:.3f} {recall=:.3f} {f1_score=:.3f}")

precision=0.576 recall=0.689 f1_score=0.628


## Threshold to penalize regions with large amount of text lines

In [17]:
adjusted_reg_score = [evaluate.adjust_score_by_dist(clip_res, len_max=5, len_penalty=0.00001, dist_far_penalty=0.00001, dist_near_threshold=10000) for clip_res in clip_region_distances]

In [18]:
adjusted_reg_results = evaluate.eval_region_data_set(data_set, adjusted_reg_score)

In [19]:
precision, recall, f1_score = evaluate.calculate_metrics(*adjusted_reg_results)
print(f"{precision=:.3f} {recall=:.3f} {f1_score=:.3f}")

precision=0.695 recall=0.798 f1_score=0.743


## Setting: sentences